## UseLib

In [102]:
############## 전처리 및 데이터 핸들링 ###############################

import numpy as np
import  pandas as pd
import os
from konlpy.tag import Okt 
okt = Okt()
from tqdm.notebook import tqdm
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import copy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


############## 머신러닝 모델#########################
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
from sklearn.svm import SVC
svc =  SVC()
from sklearn.neighbors import KNeighborsClassifier
knn =KNeighborsClassifier()
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2021)
import xgboost as xgb
from xgboost import XGBClassifier
xgc = XGBClassifier()


############## 평가 ###############################
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV

## UseData

In [8]:
cd ~/WorkSpace/GoogleDrive/제주_전기차/data ## 경로이동

/home/miwoos/WorkSpace/GoogleDrive/제주_전기차/data


In [62]:
ls ## 데이터 위치 파악 및 데이터 리스트 확인

 elc_usewordcloud.png             '전기차 동호회 전기차크롤링.csv'
 elecafe.csv                       전기차_전처리및토큰화.csv
 lr.pkl                            전기차관련검색어인기순.csv
 lrc.pkl                           전기차관심도지역.csv
 tvect.pkl                         전기차관심증가.csv
 관련키워드.csv                    전기차전처리추가.csv
 네이버-제주전기차후기통합.csv     전기차키워드관련주제인기순.csv
 느영나영전기차크롤링.csv          전체본문의댓글결과_2022.03.06.xlsx
 느영나영전기차크롤링.csv.part     제사모크롤링.csv
'본문 검색 결과_2022.03.06.xlsx'   투플럭스sjeju.csv
 빈도수파악.png


In [11]:
df_train= pd.read_csv('전기차_전처리및토큰화.csv')
df_cafe = pd.read_csv('elecafe.csv')
df_Blog = pd.read_csv('네이버-제주전기차후기통합.csv')

## 러닝

### 널값 제거 및 전처리

In [63]:
df_train.head(2)

,전처리,평가
0,아이오 닉 휠 베이스 가치 서비스 센터 가다 부품 수급 문제 로 렌터카 받다 오다 ...,P
1,현대차 아이오 닉 테슬라 모델 뭘 살다 요 현대차 아이오 닉 테슬라 모델 뭘 살다 ...,P


In [72]:
print(df_train.isnull().sum())
print(df_cafe.isnull().sum())
print(df_Blog.isnull().sum())

전처리    0
평가     0
dtype: int64
제목      0
내용      0
댓글      0
유형      0
조회수     0
작성날짜    0
월별      0
카페명     0
년별      0
dtype: int64
제목      0
날짜      0
내용      0
href    0
dtype: int64


In [21]:
del df_Blog['index'] # 인덱스칼럼이 널값이많아서 컬럼 자체를 없애 버림 
df_train.dropna(inplace=True) # 드랍으로 널값제거

### 이진분류

In [33]:
df_이진분류 = copy.deepcopy(df_train[df_train !='E'])

In [76]:
df_이진분류.head(2)
df_이진분류.dropna(inplace=True)
df_이진분류.isnull().sum()

전처리    0
평가     0
dtype: int64

In [77]:
X_train,X_test,y_train,y_test = train_test_split (
    df_이진분류['전처리'],df_이진분류['평가'],test_size=0.25,random_state=2021
)

In [78]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((218401,), (72801,), (218401,), (72801,))

##### 타겟값 라벨링

In [79]:
y_train_label = LabelEncoder().fit_transform(y_train)
y_test_label = LabelEncoder().fit_transform(y_test)

In [80]:
y_train_label,y_test_label

(array([1, 0, 1, ..., 0, 1, 0]), array([1, 1, 0, ..., 1, 1, 1]))

#### CV binary

In [98]:
cvect_binary =  CountVectorizer()
cvect_binary.fit(X_train)

CountVectorizer()

In [85]:
X_train_cv_binary = cvect_binary.transform(X_train)
X_test_cv_binary =  cvect_binary.transform(X_test)

#### TF-IDF binary

In [86]:
tvect_binary = TfidfVectorizer(ngram_range=(1,2), max_df = 0.9)

In [87]:
%time tvect_binary.fit(X_train)


CPU times: user 2min 53s, sys: 696 ms, total: 2min 53s
Wall time: 2min 53s


TfidfVectorizer(max_df=0.9, ngram_range=(1, 2))

In [ ]:
%time X_train_tv_binary = tvect_binary.transform(X_train)
%time X_test_tv_binary = tvect_binary.transform(X_test)

CPU times: user 16.1 s, sys: 123 ms, total: 16.3 s
Wall time: 16.3 s
CPU times: user 5.22 s, sys: 23.7 ms, total: 5.24 s
Wall time: 5.26 s


### 다중분류

In [89]:
XX_train,XX_test,yy_train,yy_test = train_test_split (
    df_train['전처리'],df_train['평가'],test_size=0.25,random_state=2021
)

In [90]:
XX_train.shape,XX_test.shape,yy_train.shape,yy_test.shape

((296083,), (98695,), (296083,), (98695,))

#### cv 

In [97]:
cvect  = copy.deepcopy(CountVectorizer())
cvect.fit(XX_train)

CountVectorizer()

In [ ]:
X_train_cv = cvect.transform(XX_train)
X_test_cv=  cvect.transform(XX_test)

#### TF-IDF

In [99]:
tvect = copy.deepcopy(TfidfVectorizer(ngram_range=(1,2), max_df = 0.9))

In [ ]:
%time tvect.fit(XX_train)
%time XX_train_tv = tvect.transform(XX_train)
%time XX_test_tv = tvect.transform(XX_test)


CPU times: user 3min 8s, sys: 844 ms, total: 3min 9s
Wall time: 3min 8s
CPU times: user 18.4 s, sys: 147 ms, total: 18.5 s
Wall time: 18.6 s
CPU times: user 5.86 s, sys: 35.8 ms, total: 5.9 s
Wall time: 5.9 s


## 모델 비교 평가

In [114]:
ests =[lr,nb,xgc,svc,knn,rfc]

In [ ]:
for est in ests:
    est.fit(X_train_cv_binary,y_train)
    print(est.score(X_test_cv,y))

In [116]:
def auto(model,vect,i):
    g= globals()
    if vect == 'cvect':
        g[f'{vect}{i}'] = CountVectorizer(stop_words='english',ngram_range=(1,i))
    elif vect == 'tvect':
        g[f'{vect}{i}'] = TfidfVectorizer(stop_words='english',ngram_range=(1,i))
    g[f'{vect}{i}'].fit(X_train)
    g[f'X_train_{vect}{i}'] = g[f'{vect}{i}'].transform(X_train)
    g[f'X_test_{vect}{i}'] = g[f'{vect}{i}'].transform(X_test)
    model.fit(g[f'X_train_{vect}{i}'],y_train)
    return model.score(g[f'X_test_{vect}{i}'],y_test)

In [117]:
for model in [lr,nb,xgc,svc,knn,rfc]:
    for vect in ['cvect','tvect']:
            for i in [1,2]:
                    print(f'모델:{model}\t,벡터:{vect}\tngram_range:{i}일때:{auto(model,vect,i)}입니다.')

/home/miwoos/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


모델:LogisticRegression()	,벡터:cvect	ngram_range:1일때:0.8927761981291465입니다.


/home/miwoos/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


모델:LogisticRegression()	,벡터:cvect	ngram_range:2일때:0.8896443730168542입니다.


/home/miwoos/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

모델:LogisticRegression()	,벡터:tvect	ngram_range:2일때:0.8848642188980921입니다.
모델:MultinomialNB()	,벡터:cvect	ngram_range:1일때:0.7300311808903723입니다.
모델:MultinomialNB()	,벡터:cvect	ngram_range:2일때:0.7680251644894988입니다.
모델:MultinomialNB()	,벡터:tvect	ngram_range:1일때:0.7848793285806513입니다.
모델:MultinomialNB()	,벡터:tvect	ngram_range:2일때:0.7220093130588866입니다.


/home/miwoos/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:14:56] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
모델:XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None,
              enable_categorical=False, gamma=None, gpu_id=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method=None, validate_parameters=None, verbosity=None)	,벡터:cvect	ngram_range:1일때:0.8649331739948627입니다.


/home/miwoos/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[11:18:23] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
모델:XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)	,벡터:cvect	ngram_range:2일때:0.865523825222181입니다.


In [ ]:
def auto2(model,vect,i):
    g= globals()
    if vect == 'cvect':
        g[f'{vect}{i}'] = CountVectorizer(ngram_range=(1,i))
    elif vect == 'tvect':
        g[f'{vect}{i}'] = TfidfVectorizer(ngram_range=(1,i))
    g[f'{vect}{i}'].fit(X_train)
    g[f'XX_train_{vect}{i}'] = g[f'{vect}{i}'].transform(XX_train)
    g[f'XX_test_{vect}{i}'] = g[f'{vect}{i}'].transform(XX_test)
    model.fit(g[f'XX_train_{vect}{i}'],y_train_label)
    return model.score(g[f'XX_test_{vect}{i}'],y_test_label)

In [ ]:
for model in [lr,nb,xgc,svc,knn,rfc]:
    for vect in ['cvect','tvect']:
            for i in [1,2]:
                    print(f'다중분류 모델:{model}\t,벡터:{vect}\tngram_range:{i}일때:{auto2(model,vect,i)}입니다.')

In [ ]:
for model in [lr,nb,xgc,svc,knn,rfc]:
    for test in [X_test_cv_binary,X_test_tv_binary,XX_test_cv,XX_test_tv]:
        pred=model.predict(test)
        if test in [X_test_cv_binary,X_test_tv_binary]:
            ave = 'binary'
        else:
            ave = 'macro'
        accuracy_score(y_test_label, pred)
        p = precision_score(y_test_label, pred,average=ave)
        print(model'일떄'+test'이면'+'p는'p'입니다')
        r = recall_score(y_test_label, pred,average=ave)
        print(model'일떄'+test'이면'+'r는'r'입니다')
        f1 = f1_score(y_test_label, pred,average=ave)
        print(model'일떄'+test'이면'+'p는'f1'입니다'))